In [1]:
from makiflow.layers import *
from makiflow.models.segmentation.segmentator import Segmentator
from makiflow.augmentation import AffineAugment, ElasticAugment, ImageCutter, Data
from makiflow.save_recover import Builder
from makiflow.trainers import SegmentatorTrainer
import makiflow as mf

import tensorflow as tf
import numpy as np
import glob
import cv2
import seaborn as sns
from tqdm import tqdm
from matplotlib import pyplot as plt

from sklearn.utils import shuffle
from scipy.ndimage import gaussian_filter

/home/rustam/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def load_data(path_to_data='../dataset/mask'):
    Xtrain = []
    Ytrain = []

    masks = glob.glob(f'{path_to_data}/*.bmp')
    for mask_name in tqdm(masks):
        img = cv2.imread(mask_name.replace('mask', 'imgs'))
        mask = cv2.imread(mask_name)
        Xtrain.append(img)
        Ytrain.append(mask)
        
    return Xtrain, Ytrain

In [3]:
def calc_num_pos(labels):
    area = labels[0].shape[0] * labels[0].shape[1]
    return [area - (label == 0).sum() for label in labels]

### Load data

In [4]:
images, labels = load_data()

100%|██████████| 100/100 [00:00<00:00, 191.05it/s]


In [5]:
Xtest = []
Ytest = []
Xtrain = []
Ytrain = []

for i, label in enumerate(labels):
    uniq = np.unique(label)
    if i in [25, 41, 14, 19, 43, 2, 5]:
        Xtest.append(images[i])
        Ytest.append(label)
        continue
    else:
        Xtrain.append(images[i])
        Ytrain.append(label)

In [6]:
print(len(Xtrain), len(Xtest))

93 7


# Important 
25/41/14/19/43/2/5

In [7]:
from copy import copy

In [8]:
for image, label in zip(copy(Xtrain), copy(Ytrain)):
    uniq = np.unique(label)
    if 40 in uniq or 70 in uniq or 80 in uniq:
        Ytrain += [label] * 10
        Xtrain += [image] * 10

In [9]:
Xtrain, Ytrain, _ = ImageCutter.image_and_mask_cutter(Xtrain, Ytrain, 256, 256, 128, 128, 0.5)
Xtest, Ytest, _ = ImageCutter.image_and_mask_cutter(Xtest, Ytest, 256, 256, 128, 128, 0.5)
print(len(Xtrain), len(Xtest))

13201 237


In [10]:
Xtrain = np.asarray(Xtrain).astype(np.float32) / 255
Xtrain = [i for i in Xtrain]
Ytrain = np.asarray(Ytrain).astype(np.uint8) // 10
Ytrain = [i for i in Ytrain]
Xtest = np.asarray(Xtest).astype(np.float32) / 255
Xtest = [i for i in Xtest]
Ytest = np.asarray(Ytest).astype(np.uint8) // 10
Ytest = [i for i in Ytest]

In [11]:
Ytest = [cv2.cvtColor(item, cv2.COLOR_BGR2GRAY) for item in Ytest]

In [12]:
print(Xtest[0].shape, Ytest[0].shape)

(256, 256, 3) (256, 256)


In [13]:
mf.set_main_gpu(0)
model = Builder.segmentator_from_json('unetMobileNet/model.json')


Model is restored!


In [14]:
model.set_session(tf.Session())

In [36]:
model.load_weights('result/Test_pretrained_Unet_with_MobileNetV2_backbone/MakiSegmentator_gamma=4.0_lr=0.001_bsz=32/epoch_15/weights.ckpt')

INFO:tensorflow:Restoring parameters from result/Test_pretrained_Unet_with_MobileNetV2_backbone/MakiSegmentator_gamma=4.0_lr=0.001_bsz=32/epoch_15/weights.ckpt
Weights are loaded.


In [37]:
batch_sz = 16

In [38]:
n_batches = len(Xtrain[:250]) // batch_sz
labels = []
predictions = []
for i in range(n_batches):
    labels += Ytrain[i * batch_sz: (i+1) * batch_sz]
    predictions += [model.predict(Xtrain[i * batch_sz: (i+1) * batch_sz])]

In [39]:
n_batches = len(Xtest) // batch_sz
labels = []
predictions = []
for i in range(n_batches):
    labels += Ytest[i * batch_sz: (i+1) * batch_sz]
    predictions += [model.predict(Xtest[i * batch_sz: (i+1) * batch_sz])]

In [40]:
predictions = np.vstack(predictions)
labels = np.asarray(labels)

## Calculate Dice manually

In [20]:
from makiflow.metrics import one_hot

In [53]:
v_dice_history = []
dices_history = []

In [21]:
EPSILON = 1e-9
def binary_dice(predicted, actual):
    num = np.sum(predicted * actual)
    den = np.sum(predicted * predicted) + np.sum(actual)
    return (2 * num + EPSILON) / (den + EPSILON)

In [22]:
def categorical_dice_coeff(P, L, use_argmax=False):
    """
    Calculates V-Dice for give predictions and labels.
    WARNING! THIS IMPLIES SEGMENTATION CONTEXT.
    Parameters
    ----------
    P : np.ndarray
        Predictions of a segmentator. Array of shape [batch_sz, W, H, num_classes].
    L : np.ndarray
        Labels for the segmentator. Array of shape [batch_sz, W, H]
    use_argmax : bool
        Converts the segmentator's predictions to one-hot format.
        Example: [0.4, 0.1, 0.5] -> [0., 0., 1.]
    """
    batch_sz = len(P)
    L = np.asarray(L)
    P = np.asarray(P)
    num_classes = P.shape[-1]
    if use_argmax:
        P = P.argmax(axis=3)
        P = P.reshape(-1)
        P = one_hot(P, depth=num_classes)
    P = P.reshape(batch_sz, -1, num_classes)
    L = L.reshape(batch_sz, -1)

    class_dices = np.zeros(num_classes)
    class_counts = np.zeros(num_classes)
    for i in range(batch_sz):
        sample_actual = L[i]
        sample_pred = P[i]
        for j in range(num_classes):
            sub_actual = (sample_actual[:] == j).astype(np.int32)
            sub_confs = sample_pred[:, j]
            if np.sum(sub_actual) == 0 and np.sum(sub_confs)== 0:
                continue
            class_dices[j] += binary_dice(sub_confs, sub_actual)
            class_counts[j] += 1
    return (class_dices / class_counts).mean(), class_dices / class_counts

In [41]:
v_dice, dices = categorical_dice_coeff(predictions, labels, use_argmax=True)
v_dice, dices

(0.32951139469688506,
 array([9.54269185e-01, 5.91409698e-01, 3.47100258e-01, 6.91377561e-01,
        1.10191396e-01, 1.83360695e-01, 4.71772217e-05, 2.14510450e-10,
        1.75994914e-10, 4.17357977e-01]))

In [122]:
v_dice_history += [v_dice]
dices_history += [dices]

In [123]:
from makiflow.tools import TestVisualizer

In [129]:
dices = []
for i in range(10):
    single_dice = []
    for dice_list in dices_history:
        single_dice.append(dice_list[i])
    dices += [single_dice]

In [130]:
TestVisualizer.plot_test_values(
            test_values=dices,
            legends=[
        "bkg",
        "dzn",
        "m",
        "s",
        "ns",
        "te",
        "me",
        "sc",
        "pc",
        "rg"
      ],
            x_label='Epochs',
            y_label='Dice',
            save_path=f'dices.png'
        )

In [133]:
TestVisualizer.plot_test_values(
            test_values=[v_dice_history],
            legends=['v-dice'],
            x_label='Epochs',
            y_label='Dice',
            save_path=f'v_dice.png'
        )

In [132]:
v_dice_history

[0.25735925003105875,
 0.3115492244254012,
 0.32002028018926126,
 0.3246473240606206,
 0.32033873385110223,
 0.32924083415601635,
 0.3338713259840366,
 0.33948482837793253,
 0.3321843133573052,
 0.3373498575333983,
 0.3311323268654459]